# Cross validation

Cross-validation (CV) is an important step in the whole predictive modelling process. Luckily it is not the hardest bit to code. Let's have a look at the basic setup below.

## The dataset and classifier

First, let's introduce the dataset and divide it into training and test set:

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

X,y = make_classification(n_samples=1000, n_features=10,
                               n_informative=2, n_redundant=0, n_repeated=0,
                               n_classes=2,
                               n_clusters_per_class=1,
                               weights=(0.7,0.3),
                               class_sep=0.99, random_state=14)


# You already know about training and test splits:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42)

To apply cross-validation, we need a classifier as well. Let's use logistic regression for now:

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(solver='liblinear')

## Applying cross validation

Here, we introduce the classifier into the CV process. It really is a process, e.g., for 10-Fold CV, we have a 10-step process. The classifier is embedded into the whole process, and used on the 10 training sets that are generated:

In [5]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(classifier, X_train, y_train, cv=5)
print('Accuracy scores: '+str(scores))

Accuracy scores: [0.97857143 1.         0.97857143 0.99285714 0.99285714]


What have we just calculated? In this case, the accuracy. Let's now add some other metrics as well, e.g., the AUC:

In [6]:
outcomes = cross_val_score(classifier, X_train, y_train, cv=10, scoring='roc_auc')
print(outcomes)

[1.         0.99431818 1.         1.         0.99147727 1.
 1.         1.         0.9718173  1.        ]


If you are interested in multiple metrics at the same time, another function is more appropriate:

In [7]:
from sklearn.model_selection import cross_validate

# metrics you want to have computed
metrics = ['roc_auc','accuracy','precision']

# By default, we should not really care about the training scores. To show them, we add the extra return_train_score parameter
outcomes = cross_validate(classifier, X_train, y_train, scoring=metrics, cv=10, return_train_score=True)
for metric in outcomes.keys():
    print(metric+" value: "+str(outcomes[metric]))

fit_time value: [0.00175309 0.00176263 0.00162911 0.00166655 0.00164938 0.00172567
 0.00173163 0.00167274 0.00166178 0.00169444]
score_time value: [0.00537491 0.00264978 0.00263596 0.00246501 0.00238514 0.00247788
 0.00254154 0.00267434 0.00239921 0.0024128 ]
test_roc_auc value: [1.         0.99431818 1.         1.         0.99147727 1.
 1.         1.         0.9718173  1.        ]
train_roc_auc value: [0.99679871 0.99722555 0.99684614 0.99691728 0.9973204  0.9969616
 0.99691431 0.99671333 0.99949163 0.99673697]
test_accuracy value: [0.95714286 0.98571429 1.         1.         0.97142857 1.
 1.         0.98571429 0.98571429 1.        ]
train_accuracy value: [0.99206349 0.99206349 0.98888889 0.98888889 0.99365079 0.98888889
 0.98888889 0.99047619 0.99047619 0.98888889]
test_precision value: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
train_precision value: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


Now, the outcome is a dictionary with the different metrics per fold for both the training and test set (note that, since we have set aside a separate test set, this is our validation set in this case).

## Setting up a pipeline

Remember when we talked about training, validation and test sets, we mentioned that the pre-processing (e.g., replacing missing values, transformations, over- and under-sampling, etc.) should be performed on the training and test set separately to avoid any bias? **That is, the same transformation, with the same parameters, should be applied to both.** Otherwise, information of the testing set can 'leak' into the training process, while the testing stage needs to be completely independent.

To simplify this, we can set up a pipeline containing the various steps that need to be applied, i.e., transformation and training a classifier:

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

metrics = ['accuracy']

pipeline = make_pipeline(StandardScaler(), classifier)
outcomes = cross_validate(pipeline, X_train, y_train, scoring=metrics, cv=10, return_train_score=True)
for metric in outcomes.keys():
    print(metric+" value: "+str(outcomes[metric]))

## Predictions for every sample

If you want to obtain the predictions for every sample from when it was in the test set (in 10-fold CV, every sample is used exactly once), the following code can be used:

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score

predictions = cross_val_predict(pipeline, X_train, y_train, cv=10)
print(accuracy_score(y_train, predictions))

Typically, we will use cross-validation to see what classifier, or what parameters, are working best over our training/validation sets. Then, finally, we use them on our test set for our final evaluation.

## Adding sampling strategy to pipeline

Since our data is imbalanced, we might want to preserve this imbalance in every fold. To do so, we can use the stratified CV procedure as well:

In [ ]:
from sklearn.model_selection import StratifiedKFold

stratified_kfold = StratifiedKFold(n_splits=10, shuffle= True, random_state=42)
outcomes = cross_validate(pipeline, X_train, y_train, scoring=metrics, cv=stratified_kfold, return_train_score=True)
for metric in outcomes.keys():
    print(metric+" value: "+str(outcomes[metric]))